## mvSuSiE benchmark updated

During the past few months we have implemented a few fixes with input from Yuxin who performed mvSuSiE analysis in GWAS context and ironed out some corner cases. Also progress from `udr` package offers us better estimate for mixture prior. We now rerun all the benchmark previously developed and look at updated results.

## Benchmark execution

Under the `dsc/mnm_prototype` directory,

```
sos run 20210224_MNM_Benchmark simulation_only
sos run 20210224_MNM_Benchmark get_prior
```


## Workflow implementations

In [ ]:
[simulation_only]
script: interpreter= 'qsub', expand = True
#!/bin/bash

#SBATCH --time=36:00:00
#SBATCH --partition=broadwl
#SBATCH --nodes=1
#SBATCH --ntasks-per-node=1
#SBATCH --cpus-per-task=8
#SBATCH --mem-per-cpu=4000
#SBATCH --job-name={step_name}
#SBATCH --mail-type=BEGIN,END,FAIL

module load R
./gtex_qtl.dsc --host dsc_mnm.yml --target simulate_only --n_dataset 25000 -o mnm_sumstats -s existing -e ignore &> mnm_sumstats.log

In [ ]:
[get_prior]
#download: dest_file = 'mixture_prior.ipynb'
#    https://raw.githubusercontent.com/cumc/bioworkflows/master/multivariate-fine-mapping/mixture_prior.ipynb

script: interpreter= 'qsub', expand = True
#!/bin/bash
  
#SBATCH --time=36:00:00
#SBATCH --partition=broadwl
#SBATCH --nodes=1
#SBATCH --ntasks-per-node=1
#SBATCH --cpus-per-task=8
#SBATCH --mem-per-cpu=2000
#SBATCH --job-name={step_name}
#SBATCH --mail-type=BEGIN,END,FAIL

module load R
# 1. extract data from `simulate_only` pipeline in DSC 
# `analysis_units.txt` is prepared by `ls `
for m in mnm_sumstats/artificial_mixture_identity mnm_sumstats/gtex_mixture_identity; do
    cd $m && ls *.rds | sed 's/\.rds//g' > analysis_units.txt && cd -
    sos run mixture_prior.ipynb extract_effects \
        --analysis-units $m/analysis_units.txt \
        --datadir $m --name `basename $m` \
        -c ../../midway2.yml -q midway2 -s build
done
# 2. Run FLASH for data extracted
#for m in artificial_mixture_identity gtex_mixture_identity; do
#sos run ~/GIT/gtexresults/workflows/mashr_flashr_workflow.ipynb flash \
#    --cwd /project2/mstephens/gaow/mvarbvs/dsc/mnm_prototype/mnm_sumstats/ \
#    --data /project2/mstephens/gaow/mvarbvs/dsc/mnm_prototype/mnm_sumstats/$m.rds \
#    --effect-model EE -c midway2.yml -q midway2
#done
# 3. Run ED for artificial mixture
# sos run analysis/20200502_Prepare_ED_prior.ipynb ed --model artificial_mixture_identity -c midway2.yml -q midway2
# 4. Run ED for GTEx mixture
# sos run analysis/20200502_Prepare_ED_prior.ipynb ed --model gtex_mixture_identity -c midway2.yml -q midway2


In [ ]:
# Run the main DSC
dsc ukb_rss.dsc --host dsc_mnm.yml -o output/ukb_rss -s existing &> mnm_rss_ukb.log
# Run the mthess DSC
#./fixed_mix.dsc --host dsc_mnm.yml --target mthess -o mthess_20200526 -s existing -e ignore --n_dataset 200 &> mnm_20200526.log
# Run data generation step only to get data for ED estimate
# dsc ukb_rss.dsc --host dsc_mnm.yml --target simulate_only -s existing -o output/ukb_rss &> mnm_rss_ukb.log

# Run the main DSC
#./fixed_mix.dsc --host dsc_mnm.yml -o mnm_20200510 -s existing -e ignore &> mnm_20200510.log
# Run the mthess DSC
./fixed_mix.dsc --host dsc_mnm.yml --target mthess -o mthess_20200526 -s existing -e ignore --n_dataset 200 &> mnm_20200526.log
# Run data generation step only to get data for ED estimate
#./fixed_mix.dsc --host dsc_mnm.yml --target simulate_only --n_dataset 25000 -o mnm_sumstats -s existing -e ignore &> mnm_sumstats.log